In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "0733b351",
   "metadata": {},
   "source": [
    "# AIoT Project"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4455947d",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "\n",
    "# basic data engineering\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import scipy\n",
    "\n",
    "# plotting\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "# db\n",
    "import pymongo\n",
    "\n",
    "# configs & other\n",
    "import yaml\n",
    "from tqdm.notebook import tqdm_notebook\n",
    "from datetime import datetime\n",
    "from time import time\n",
    "\n",
    "from psynlig import pca_explained_variance_bar\n",
    "\n",
    "# utils processing\n",
    "from utils import sliding_window_pd\n",
    "from utils import apply_filter\n",
    "from utils import filter_instances\n",
    "from utils import flatten_instances_df\n",
    "from utils import df_rebase\n",
    "from utils import rename_df_column_values\n",
    "\n",
    "# utils visualization\n",
    "from utils_visual import plot_instance_time_domain\n",
    "from utils_visual import plot_instance_3d\n",
    "from utils_visual import plot_np_instance\n",
    "from utils_visual import plot_heatmap\n",
    "from utils_visual import plot_scatter_pca\n",
    "\n",
    "%load_ext autoreload\n",
    "%autoreload 2"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "07adb678",
   "metadata": {},
   "source": [
    "Start time of execution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "717559c4",
   "metadata": {},
   "outputs": [],
   "source": [
    "time_start = time()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "34b96e6f",
   "metadata": {},
   "source": [
    "## Load configuration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "982eac13",
   "metadata": {},
   "outputs": [],
   "source": [
    "config_path = os.path.join(os.getcwd(), \"config.yml\")\n",
    "\n",
    "with open(config_path) as file:\n",
    "    config = yaml.load(file, Loader=yaml.FullLoader)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bfce0bca",
   "metadata": {},
   "outputs": [],
   "source": [
    "client = pymongo.MongoClient(config[\"client\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dc950bbb",
   "metadata": {},
   "outputs": [],
   "source": [
    "db = client[config[\"db\"]]\n",
    "coll = db[config[\"col\"]]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2221ec1a",
   "metadata": {},
   "outputs": [],
   "source": [
    "found_keys = coll.distinct(\"label\")\n",
    "print(\"Existing DB keys:\", found_keys)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0c2b7f9c",
   "metadata": {},
   "source": [
    "## Apply filter"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f73e077d",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample code to apply filters on data fetched from MongoDB\n",
    "data_cursor = coll.find()\n",
    "data_list = list(data_cursor)\n",
    "\n",
    "data_df_list = [pd.DataFrame(doc['data']) for doc in data_list]\n",
    "\n",
    "filtered_data = filter_instances(data_df_list, order=config['filter']['order'], wn=config['filter']['wn'], filter_type=config['filter']['type'])"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "174424cd",
   "metadata": {},
   "source": [
    "## Transform the list of DataFrames to NumPy array\n",
    "\n",
    "Transform the list of DataFrames to NumPy array that contains the windows: (instances, x, y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "741fe8b6",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample code to transform DataFrames to NumPy arrays\n",
    "np_data_list = [df.to_numpy() for df in filtered_data]\n",
    "\n",
    "# Assuming `sliding_window_pd` is already defined in utils, apply it here\n",
    "windows_list = [sliding_window_pd(df, ws=config['sliding_window']['ws'], overlap=config['sliding_window']['overlap']) for df in np_data_list]\n",
    "\n",
    "# Flatten the list of lists\n",
    "windows = [window for sublist in windows_list for window in sublist]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9587c7d9",
   "metadata": {},
   "source": [
    "## Flatten the 2D window instances\n",
    "\n",
    "Flatten the X NumPy array that contains the 2D window instances"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c49d38ed",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample code to flatten the window instances\n",
    "X = flatten_instances_df(windows)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "41620e98",
   "metadata": {},
   "source": [
    "## Train/Test split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1d592023",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b2b23d31",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample labels\n",
    "y = [doc['label'] for doc in data_list]\n",
    "\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "a3b6aded",
   "metadata": {},
   "source": [
    "## Scaling"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b12737cf",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.preprocessing import StandardScaler, MinMaxScaler"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "7d3d85a8",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample code for scaling\n",
    "scaler = StandardScaler()\n",
    "X_train_scaled = scaler.fit_transform(X_train)\n",
    "X_test_scaled = scaler.transform(X_test)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "bb41accf",
   "metadata": {},
   "source": [
    "## Dimensionality Reduction with PCA using the 1D (flattened) data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c0c8c60a",
   "metadata": {},
   "outputs": [],
   "source": [
    "# add transformers\n",
    "from sklearn.decomposition import PCA"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "754f3a1b",
   "metadata": {},
   "outputs": [],
   "source": [
    "# PCA with 2 components\n",
    "pca = PCA(n_components=2)\n",
    "X_train_pca = pca.fit_transform(X_train_scaled)\n",
    "X_test_pca = pca.transform(X_test_scaled)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8dd95258",
   "metadata": {},
   "source": [
    "### PCA with 2 Components"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4d8c1252",
   "metadata": {},
   "outputs": [],
   "source": [
    "pca2d = PCA(n_components=2)\n",
    "pca2d.fit(X_train_scaled)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1fd25884",
   "metadata": {},
   "outputs": [],
   "source": [
    "X_train_2d = pca2d.transform(X_train_scaled)\n",
    "X_test_2d = pca2d.transform(X_test_scaled)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1bc0ea67",
   "metadata": {
    "scrolled": false
   },
   "outputs": [],
   "source": [
    "pca_explained_variance_bar(pca2d, alpha=0.8)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ee7b3f0f",
   "metadata": {},
   "outputs": [],
   "source": [
    "plot_scatter_pca(X_train_2d, y_train, title='PCA 2D Scatter Plot')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "91e107a4",
   "metadata": {},
   "source": [
    "### PCA with 3 Components"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2f59fb62",
   "metadata": {},
   "outputs": [],
   "source": [
    "pca3d = PCA(n_components=3)\n",
    "pca3d.fit(X_train_scaled)\n",
    "X_train_3d = pca3d.transform(X_train_scaled)\n",
    "X_test_3d = pca3d.transform(X_test_scaled)\n",
    "\n",
    "plot_instance_3d(X_train_3d, y_train)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "87e8d62f",
   "metadata": {},
   "source": [
    "### PCA with X% of the variance of the dataset, for training the statistical AI Models"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "743bcd58",
   "metadata": {},
   "outputs": [],
   "source": [
    "pca_var = PCA(n_components=0.95)  # Adjust the variance percentage as needed\n",
    "X_train_var = pca_var.fit_transform(X_train_scaled)\n",
    "X_test_var = pca_var.transform(X_test_scaled)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "de03fd48",
   "metadata": {},
   "source": [
    "## Classifier - Statistical Learning"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "789ce0a8",
   "metadata": {},
   "source": [
    "### Apply simple classifier"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8846c809",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.svm import SVC\n",
    "from sklearn.ensemble import RandomForestClassifier"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c599deb1",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize and train the SVC classifier\n",
    "svc = SVC(kernel='rbf', C=config['classifier']['SVC']['C'], gamma=config['classifier']['SVC']['gamma'])\n",
    "svc.fit(X_train_var, y_train)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e2b4b498",
   "metadata": {},
   "source": [
    "### Evaluate simple classifier"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ce02a18e",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "11536a59",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate the classifier\n",
    "y_pred = svc.predict(X_test_var)\n",
    "cm = confusion_matrix(y_test, y_pred)\n",
    "disp = ConfusionMatrixDisplay(confusion_matrix=cm)\n",
    "disp.plot()\n",
    "plt.show()\n",
    "\n",
    "print(classification_report(y_test, y_pred))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ee969415",
   "metadata": {},
   "source": [
    "### Apply optimization with Grid Search and Cross-validation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e9a3c651",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import GridSearchCV"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ba17bf58",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define parameter grid\n",
    "param_grid = config['fine_tune']['param_grid']\n",
    "\n",
    "# Initialize GridSearchCV\n",
    "grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=config['fine_tune']['cv'], verbose=config['fine_tune']['verbose'])\n",
    "\n",
    "# Fit the model\n",
    "grid_search.fit(X_train_var, y_train)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0b1db5c5",
   "metadata": {},
   "source": [
    "### Evaluate optimized classifier"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9e11ff71",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate the optimized classifier\n",
    "best_svc = grid_search.best_estimator_\n",
    "y_pred_optimized = best_svc.predict(X_test_var)\n",
    "cm_optimized = confusion_matrix(y_test, y_pred_optimized)\n",
    "disp_optimized = ConfusionMatrixDisplay(confusion_matrix=cm_optimized)\n",
    "disp_optimized.plot()\n",
    "plt.show()\n",
    "\n",
    "print(classification_report(y_test, y_pred_optimized))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "95ada6c9",
   "metadata": {},
   "source": [
    "## Classifier - Neural Network"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "63cd8abe",
   "metadata": {},
   "outputs": [],
   "source": [
    "import tensorflow as tf\n",
    "from tensorflow.keras import Sequential\n",
    "from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "983c3269",
   "metadata": {},
   "outputs": [],
   "source": [
    "input_data_shape = X_train_2d[0].shape\n",
    "print(\"Type of the input shape object:\", type(input_data_shape))\n",
    "X_train_2d[0].shape"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "44bb9bab",
   "metadata": {},
   "outputs": [],
   "source": [
    "y_np_array = np.array(y)\n",
    "n_outputs = len(np.unique(y_np_array))\n",
    "print(\"Number of outputs (classes) the model to predict:\", n_outputs)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "85b5be53",
   "metadata": {},
   "source": [
    "### Create the Neural Network (NN) Architecture and instantiate the model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d2ca897e",
   "metadata": {},
   "outputs": [],
   "source": [
    "model = Sequential()\n",
    "\n",
    "# Add layers to the model\n",
    "model.add(Dense(64, activation='relu', input_shape=input_data_shape))\n",
    "model.add(Dropout(0.5))\n",
    "model.add(Dense(32, activation='relu'))\n",
    "model.add(Dense(n_outputs, activation='softmax'))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5d086695",
   "metadata": {},
   "source": [
    "Plot the Architecture of the TensorFlow model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ce534b3e",
   "metadata": {},
   "outputs": [],
   "source": [
    "tf.keras.utils.plot_model(model, show_shapes=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cbb3c78a",
   "metadata": {},
   "source": [
    "Plot the summary of the TensorFlow model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "10a1370a",
   "metadata": {},
   "outputs": [],
   "source": [
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5ffb0f58",
   "metadata": {},
   "source": [
    "### Build the NN model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "62ec9310",
   "metadata": {},
   "outputs": [],
   "source": [
    "model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "31a646d6",
   "metadata": {},
   "outputs": [],
   "source": [
    "from utils import encode_labels"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "7f39477f",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Encode labels\n",
    "y_train_encoded = encode_labels(y_train)\n",
    "y_test_encoded = encode_labels(y_test)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "105b799d",
   "metadata": {},
   "source": [
    "### Train the NN model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "595a3664",
   "metadata": {},
   "outputs": [],
   "source": [
    "history = model.fit(X_train_2d, y_train_encoded, epochs=config['fit']['epochs'], batch_size=config['fit']['batch'], validation_split=0.2, verbose=2)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ee61e2f4",
   "metadata": {},
   "source": [
    "### Evaluate the model on the test data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "ddec8b5b",
   "metadata": {},
   "outputs": [],
   "source": [
    "test_loss, test_acc = model.evaluate(X_test_2d, y_test_encoded)\n",
    "print(f\"Test accuracy: {test_acc:.4f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "46cf5b35",
   "metadata": {},
   "source": [
    "### Plot and interpret the learning curves: Loss and Accuracy based on the training and validation sets"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "df775152",
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 4))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(history.history['loss'], label='Train Loss')\n",
    "plt.plot(history.history['val_loss'], label='Val Loss')\n",
    "plt.xlabel('Epochs')\n",
    "plt.ylabel('Loss')\n",
    "plt.legend()\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(history.history['accuracy'], label='Train Accuracy')\n",
    "plt.plot(history.history['val_accuracy'], label='Val Accuracy')\n",
    "plt.xlabel('Epochs')\n",
    "plt.ylabel('Accuracy')\n",
    "plt.legend()\n",
    "\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a0e06c98",
   "metadata": {},
   "outputs": [],
   "source": [
    "# End time of execution\n",
    "time_end = time()\n",
    "print(f\"Total execution time: {time_end - time_start:.2f} seconds\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.16"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
